In [1]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
gen = ImageDataGenerator(rotation_range = 20,
                         shear_range = 0.2,
                         width_shift_range=0.2,
                         height_shift_range=0.2,
                         horizontal_flip=True)

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype(np.float32) / 255.0 #훈련 이미지 정규화
x_test = x_test.astype(np.float32) / 255.0 #테스트 이미지 정규화

In [4]:
augment_ratio = 1.5
augment_size = int(augment_ratio * x_train.shape[0])

print(augment_size)

# 전체 x_train 개수의 150% 비율만큼
randidx = np.random.randint(x_train.shape[0], size=augment_size)

# 임의로 선택된 데이터는 원본데이터를 참조하기 때문에
# 원본데이터에 영향을 줄 수 있음. 그래서 copy() 함수를 통해 안전하게 복사본 만듬
x_augmented = x_train[randidx].copy()
y_augmented = y_train[randidx].copy()

print(x_augmented.shape, y_augmented.shape)

# 이미지 보강 실행
x_augmented, y_augmented = gen.flow(x_augmented, y_augmented,
                                   batch_size=augment_size,
                                   shuffle=False).next()

print(x_augmented.shape, y_augmented.shape)


75000
(75000, 32, 32, 3) (75000, 1)
(75000, 32, 32, 3) (75000, 1)


In [5]:
# x_train, y_train 에 보강된 데이터 추가

x_train = np.concatenate( (x_train, x_augmented) )
y_train = np.concatenate( (y_train, y_augmented) )

print(x_train.shape, y_train.shape)

(125000, 32, 32, 3) (125000, 1)


In [6]:
# CNN 모델 구축

cnn = Sequential()

cnn.add(Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(32,32,3)))
cnn.add(Conv2D(32, (3,3), activation='relu', padding='same'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))

cnn.add(Conv2D(64, (3,3), activation='relu', padding='same'))
cnn.add(Conv2D(64, (3,3), activation='relu', padding='same'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))

cnn.add(Conv2D(128, (3,3), activation='relu', padding='same'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))

cnn.add(Conv2D(128, (3,3), activation='relu', padding='same'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))

cnn.add(Conv2D(256, (3,3), activation='relu', padding='same'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))

cnn.add(Flatten())

cnn.add(Dense(128, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(10, activation='softmax'))

In [8]:
from datetime import datetime

start_time = datetime.now()

cnn.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

hist = cnn.fit(x_train, y_train, batch_size=256, epochs=50, validation_data=(x_test, y_test))
# 원래 epochs = 250
end_time = datetime.now()

print('Elapsed Time => ' , end_time-start_time)

Epoch 1/50
489/489 [==============================] - 303s 618ms/step - loss: 1.5214 - accuracy: 0.4460 - val_loss: 1.3301 - val_accuracy: 0.5127
Epoch 2/50
489/489 [==============================] - 301s 616ms/step - loss: 1.2614 - accuracy: 0.5501 - val_loss: 0.9677 - val_accuracy: 0.6566
Epoch 3/50
489/489 [==============================] - 301s 615ms/step - loss: 1.1103 - accuracy: 0.6087 - val_loss: 0.8872 - val_accuracy: 0.6871
Epoch 4/50
489/489 [==============================] - 300s 614ms/step - loss: 1.0224 - accuracy: 0.6444 - val_loss: 0.8639 - val_accuracy: 0.6963
Epoch 5/50
489/489 [==============================] - 301s 615ms/step - loss: 0.9546 - accuracy: 0.6695 - val_loss: 0.7776 - val_accuracy: 0.7268
Epoch 6/50
489/489 [==============================] - 301s 615ms/step - loss: 0.9052 - accuracy: 0.6878 - val_loss: 0.7746 - val_accuracy: 0.7315
Epoch 7/50
489/489 [==============================] - 305s 624ms/step - loss: 0.8598 - accuracy: 0.7042 - val_loss: 0.6740 -

In [ ]:
cnn.evaluate(x_test, y_test)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Accuracy Trend')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='best')
plt.grid()
plt.show()

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Loss Trend')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='best')
plt.grid()
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

plt.figure(figsize=(6,6))

predicted_value = cnn.predict(x_test)

cm = confusion_matrix(y_test,
                     np.argmax(predicted_value, axis=1))
sns.heatmap(cm, annot=True, fmt='d')
plt.show()